In [ ]:
# for geodesic calculations
!pip install haversine

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from haversine import haversine, Unit
import pandas as pd
import decimal
import tqdm

In [ ]:
gdf_london = pd.read_csv('datasets_out/london_cs_data.csv') # charging station locations data
gdf_london_other = pd.read_csv('datasets_out/MyDrive/other_amenities.csv') # amenities data

In [ ]:
gdf_london

,Unnamed: 0,AddressInfo,OperatorInfo,latitude,longitude
0,503,"{'ID': 4133, 'Title': 'Glyn Hopkin Nissan', 'A...","{'ID': 7, 'Title': 'Nissan UK Dealer Network',...",51.682561,0.001116
1,547,"{'ID': 4359, 'Title': 'Masterpark Pimlico Car ...","{'ID': 1, 'Title': '(Unknown Operator)', 'Webs...",51.488485,-0.143428
2,548,"{'ID': 4363, 'Title': 'Vincent Square', 'Addre...","{'ID': 1, 'Title': '(Unknown Operator)', 'Webs...",51.493841,-0.134172
3,549,"{'ID': 4364, 'Title': 'Francis Street', 'Addre...","{'ID': 32, 'Title': 'BP Pulse', 'WebsiteURL': ...",51.495886,-0.137344
4,550,"{'ID': 4367, 'Title': 'NCP Cadogan Place', 'Ad...","{'ID': 32, 'Title': 'BP Pulse', 'WebsiteURL': ...",51.496370,-0.157076
...,...,...,...,...,...
3728,131869,"{'ID': 188185, 'Title': 'Orphanage Road', 'Add...","{'ID': 8, 'Title': 'Chargemaster', 'WebsiteURL...",51.661476,-0.391724
3729,131870,"{'ID': 188186, 'Title': 'Theydon Bois', 'Addre...","{'ID': 8, 'Title': 'Chargemaster', 'WebsiteURL...",51.671658,0.102741
3730,132255,"{'ID': 188571, 'Title': 'Premier Inn London En...","{'ID': 150, 'Title': 'The GeniePoint Network (...",51.675332,-0.019660
3731,132256,"{'ID': 188572, 'Title': 'Premier Inn London En...","{'ID': 150, 'Title': 'The GeniePoint Network (...",51.675709,-0.020354


In [ ]:
gdf_london_other

,element_type,osmid,name,geometry,college,food_court,hospital,parking,restaurant,university,latitude,longitude
0,node,451153,Central Restaurant,POINT (-0.1935029 51.6020306),0,0,0,0,1,0,NaN,NaN
1,node,12185812,Heston Services Eastbound,POINT (-0.3884515 51.4888225),0,0,0,1,0,0,NaN,NaN
2,node,13893285,NaN,POINT (-0.0391511 51.4249457),0,0,0,1,0,0,NaN,NaN
3,node,13893291,NaN,POINT (-0.0386701 51.4253916),0,0,0,1,0,0,NaN,NaN
4,node,17345465,NaN,POINT (-0.0644718 51.6518261),0,0,0,1,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
5980,node,10100610124,NaN,POINT (0.0664892 51.4887177),0,0,0,1,0,0,NaN,NaN
5981,node,10101213926,Nando's,POINT (-0.1137773 51.3563644),0,0,0,0,1,0,NaN,NaN
5982,node,10105926114,Taksim,POINT (-0.0147866 51.4731125),0,0,0,0,1,0,NaN,NaN
5983,node,10111612429,Haute Dolci,POINT (-0.3391611 51.5813745),0,0,0,0,1,0,NaN,NaN


In [ ]:
lat = []
lon = []
for i in gdf_london_other['geometry']:
  longitude, latitude = i[7:].split(' ')
  latitude = decimal.Decimal(latitude[:-1])
  longitude = decimal.Decimal(longitude)
  lat.append(latitude)
  lon.append(longitude)

In [ ]:
gdf_london_other['latitude'] = pd.Series(lat)
gdf_london_other['longitude'] = pd.Series(lon)

In [ ]:
gdf_london_other.to_csv('/content/drive/MyDrive/only_amenities.csv')

In [ ]:
# merging the 2 datasets based on distance

# new dataframe for storing the merged rows
combined = pd.DataFrame()
combined['latitude'] = ''
combined['longitude'] = ''
combined['college'] = ''
combined['food_court'] = ''
combined['hospital'] = ''
combined['parking'] = ''
combined['restaurant'] = ''
combined['university'] = ''
combined['distance'] = ''
combined['charging_station'] = ''

store_idx_charging_station_amenity_100 = []

for index, ev in gdf_london.iterrows():
    # gps coordinates of charging station
    loc2 = (ev["latitude"], ev["longitude"])

    for i, j in gdf_london_other.iterrows():
       # gps_coordinates of amenity
      loc1 = (j["latitude"], j["longitude"])
      
      # calculating distance between the 2 locations
      dist = haversine(loc1, loc2, unit = 'm')
      
      # if distance less than 100 m, add to the new table
      if dist <= 100:
        new_entry = {
            'latitude' : loc2[0],
            'longitude' : loc2[1],
            'college' : j['college'],
            'food_court': j['food_court'],
            'hospital' : j['hospital'],
            'parking' : j['parking'],
            'restaurant' : j['restaurant'],
            'university' : j['university'],
            'distance' : dist,
            'charging_station': 1
        }
        idx = (index, i)

        store_idx_charging_station_amenity_100.append(idx)
   
        combined = combined.append(new_entry, ignore_index = True)
    
      print(dist, index)

Streaming output truncated to the last 5000 lines.
19347.520338526712 656
12851.551316999525 656
22594.655854160897 656
29279.67249104803 656
22142.264044072843 656
19230.635129285758 656
15381.536621770283 656
15510.597859864487 656
18837.804110050256 656
20318.202152277474 656
15222.115746125417 656
15286.172268050492 656
21306.877574862196 656
24978.53854675021 656
24197.706017639448 656
12922.221479198211 656
20458.541789396884 656
23927.68282411917 656
11334.211258803714 656
20244.402669620686 656
14569.334538031299 656
14607.389670163302 656
29286.100082175137 656
24529.612849746605 656
41730.94959134253 656
28899.21306059776 656
16344.305868775247 656
18398.880478671625 656
20445.999340941922 656
16923.62046098565 656
12391.068249276394 656
30241.83102020492 656
34456.749447529226 656
26164.804464666733 656
21467.15037364036 656
16525.61664823276 656
28039.76988463592 656
41503.87907598463 656
20656.144678618693 656
20935.342925776953 656
33224.46708461036 656
32988.2305943146 6

KeyboardInterrupt: ignored

In [ ]:
combined.to_csv('datasets_out/comb.csv')